<a href="https://colab.research.google.com/github/Satyarth10/Mini-Project/blob/main/DS_MINOR_APRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1 - Buliding The CNN

## Importing the Keras Libraries and Packages

In [ ]:
import tensorflow
import keras
from keras.models import Sequential # We will use this to initialize our Neural Netowrk
# Since we are using images we use 2D, unlike videos which are 3D with time
from keras.layers import Convolution2D # Convolution 2D is the package that wil help us in making the first step of the CNN i.e the Convolution Step in which we add the COnvolutional Layers
from keras.layers import MaxPooling2D # This package will help us to proceed to Step 2, ThePooling Step, that will add our Pooling layers
from keras.layers import Flatten # This is the package that we will use in Step 3, Flattening in which we convert all the Pooled Feature maps build from the previous 2 steps, into a large feature vector that then becomes the input layer of our fully connected layers 
from keras.layers import Dense # This is the package we will used to add the fully connected layers in the CNN, like in ANN

## Initalising the CNN

In [ ]:
classifier = Sequential()

## Step 1 - Convolution

In [ ]:
# We create many feature maps to create the Convolutional Layer
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# filters, rows, columns - The number of filters, the number of feature detectors that we apply to get the same number of feature maps. Along with their dimmensions
# 32,3,3 means that we are going to use 32 feature detectors having a 3X3 dimmension, where each feature detector will go through the Input image to create a feature map.
# Convolutional layer will comprise of Total of 32 Feature Maps
# We will use 32 as it is generally taken for CNN
# border_mode = 'same' default value
# input_shape - Shape of our Input Image on which we will apply the Feature Detectors through the COnvolution operators to get the various Feature Maps
# B/WImage - 1 channel, 2D array, RGB(Colored) - 3 channels, 3D array (Each channel corresponds to 1 color, Red, Green or Blue)
# And each channel corresponds to one 2D array that contains the pixels of our images
# 3 - RGB, 2 - B/W
# 64 X 64 - Dimmension of the 2D array (pixels)
# Since we are Tensorflow backend - input_shape = (64,64,3)
# Theano Backend - input_shape = (3,64,64)
# Activation - ReLU as we want to increase Non-Linearity (Rectifier Linear Unit)(Rectifier Activation)

## Step 2 - Max Pooling

In [ ]:
# We take the maximum with a stride of 2, Max Pooling
# For odd dimmension of Feature map - Dimmension of Pooled Feature map : n+1 X n+1, For even dimmension of Feature map - Dimmension of Pooled Feature map : n X n
# We apply Max Pooling on each and every Feature Map
# We apply Max Pooling to reduce the size, spatial invarince, avoids overfitting and we are keeping the important features, reduces time complexity as well
classifier.add(MaxPooling2D(pool_size = (2,2)))
# pool_size = (2,2) is the optimal dimmension

## Step 3 - Flattening

In [ ]:
# Input layer of Future ANN
# By creating the Feature Maps we extracted the spatial structure informations, by getting high numbers in the Feature Maps, because of the Feature Detectors
# The high numbers in the feature maps are associated to a specific feature in the input image
# And after applying Max Pooling we still keep all these high numbers, because we take the max.
# Flattening step consists of putting all the numbers in the cells of the feature maps into a single vector
# The spatial structure information gets stored in the huge single 1D vector
# If we directly flatten the input image pixels into the huge single 1D vector, then each node of this huge vector will represent one pixel of the image independent of the pixels around it.
# So we only get informations about the pixel itself and not about how the pixel is spatially connected to the other pixels around it. Thus we don't get anyinformation about the spatial structure around this pixel.
# That is why we need to follow all the previous steps
classifier.add(Flatten())

## Step 4 - Fully Connected Layers (ANN) Full Connection

In [ ]:
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
# We have a lot of input nodes, we should'nt choose a number that is too small or too big. units is seclected by trying on diff values
# units = Common practice is to pick a power of 2

## Compiling the CNN

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# loss = It corresponds to the logarithmic class of Cost Function(Logistic), 'binary_crossentropy' for binary outcomes, Here cat or dog
# if we have more than two like cat, dog, and birds etc, then we will choose categorical_crossentropy

## Part 2 - Fitting the CNN to out Images
## Image Preprocessing Step

#### What is Image Augumentation? and How does it prevent Overfitting?

We know that one of the situations that lead to overfitting is when we have few data to train our model, in that situation our model can find some correlatios in the few observations of the training set, but fails to generalize these correlations on some new observations, And when it comes to images we actually need a lot of images to generalize some correlations, Because in Computer Vision, the ML model needs to find some patterns in the pixels and to do this it requires a lot of images.

If we don't do Image Augumentation, what we might get is a result with great accuracy on the training set but a much lower accuracy on the test set

Right now, we are working with 10,000 images, 8000 for training set and 2000 for test set, and that is actually not much to obtain a great performance and that is where we will use Image Augumentation.


It basicallly creates many batches of our images, and then on each batch it applies some transformations on a random selection of our image, like rotating them, flipping them, snipping them or even shearing them. Thus we will get a lot of diverse images inside these batches and therefore a lot more material to train. And this is why it is called Image Augumentation.
The model will never find same picture across the batches as it is random.

### It is a technique that allows to enrich our datset, our training set without adding more images thus allowing us to get good performance results with little or no overfitting even with a small amount of images.

In [ ]:
# We will take help from the Keras Documentation
# First function we will use is to generate Image Augumentation: ImageDataGenerator,
# There are two ways: .flow(x, y):, .flow_from_directory(directory):
# We will use .flow_from_directory(directory): # As we have a directory to import the dateset from.
from keras.preprocessing.image import ImageDataGenerarto

train_datagen = ImageDataGenerator(
        rescale=1./255,   # feature scaling(0 and 1)  ############
        shear_range=0.2,  # Shearing (also called transvection)  # Applying several transformations, {Image Augumentation}
        zoom_range=0.2,   # random zoom                          # on Training Set
        horizontal_flip=True)    # Flip               ############

test_datagen = ImageDataGenerator(rescale=1./255)   # Only rescaling on Test Set

# Creates the training set and test set
training_set = train_datagen.flow_from_directory(r'D:\dataset\training_set',
                                                 target_size=(64, 64),  # Size of images expected in CNN Model
                                                 batch_size=1,     # Size of our bathces, after which weights will get updated
                                                 class_mode='binary')  # Types of classes
                                                    
test_set = test_datagen.flow_from_directory(r'D:\dataset\test_set',
                                            target_size=(64, 64),  # Size of images expected in CNN Model
                                            batch_size=1,     # Size of our bathces, after which weights will get updated
                                            class_mode='binary')  # Types of classes
        

# Fits the CNN model on the training set as well as testing its performance on the test set
classifier.fit_generator(training_set,
                        steps_per_epoch=8000,  # Number of images we have in our training set (Since we have 8000 training set data)
                        epochs=25,   # Number of epochs
                        validation_data=test_set,
                        validation_steps=2000)  # Number of images we have in our test set (Since we have 2000 training set data)
        

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
8000/8000 [==============================] - 404s 51ms/step - loss: 0.6865 - accuracy: 0.5573 - val_loss: 0.6738 - val_accuracy: 0.5880
Epoch 2/25
8000/8000 [==============================] - 337s 42ms/step - loss: 0.6401 - accuracy: 0.6426 - val_loss: 0.6423 - val_accuracy: 0.6490
Epoch 3/25
8000/8000 [==============================] - 261s 33ms/step - loss: 0.6167 - accuracy: 0.6678 - val_loss: 0.5968 - val_accuracy: 0.6890
Epoch 4/25
8000/8000 [==============================] - 260s 32ms/step - loss: 0.6010 - accuracy: 0.6764 - val_loss: 0.5898 - val_accuracy: 0.6940
Epoch 5/25
8000/8000 [==============================] - 281s 35ms/step - loss: 0.5886 - accuracy: 0.6927 - val_loss: 0.6403 - val_accuracy: 0.6770
Epoch 6/25
8000/8000 [==============================] - 288s 36ms/step - loss: 0.5721 - accuracy: 0.7059 

KeyboardInterrupt: 

## With 1 Convolutional layer we got a Training set accuracy of 81.42% and a Test set accuracy of 71.60%

## In order to increase our accuracy:
1. Add another Convolutional Layer (Best) {After Max Pooling Step}
2. Add another Fully Connected Layer

#### Adding a 2nd Convolutional Layer to improve the accuracy
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))  # No need for input_shape as keras knows

classifier.add(MaxPooling2D(pool_size = (2,2))) # pool_size = (2,2) is the optimal dimmension

If we add another Convolutional Layer we can double the number of feature detectors, instead of 32 we can talk 64.

## Part 3 - Predicting new Values

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('D:\ML Python\P14-Part8-Deep-Learning\Section 36 - Convolutional Neural Networks\dataset\single_prediction\cat_or_dog_2.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)   # Used to add the 4th dimmension, This new dimmension corresponds to the batch
result = classifier.predict(test_image)  # We need 4 Dimenssions insted of 3D
# In general the functions of neural networks cannot accept a single input by itself, like in this case for a single image,
# It always accepts inputs in a batch, even if batch = 1 or several inputs
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'

In [ ]:
result

array([[0.1570709]], dtype=float32)

In [ ]:
prediction

'Cat'